In [ ]:
%load_ext autoreload

%autoreload 2

# Exploration du dataset
- Aperçu général: colonnes, valeurs manquantes, duplicats, types de données
- Vérification et nettoyage minimal (si nécéssaire)

In [ ]:
# |exporti
import streamlit as st
import pandas as pd
from streamlit_jupyter import StreamlitPatcher, tqdm

StreamlitPatcher().jupyter()

st.title("Analyse des contenus Netflix")

st.header("Exploration du dataset")

df = pd.read_csv("data/netflix_titles.csv")

# Afficher les colonnes
st.subheader("Colonnes :")
st.write(", ".join([c.title() for c in df.columns]))

# Compter les valeurs manquantes par colonne
st.subheader("Valeurs manquantes par colonne :")
st.dataframe(df.isna().sum().to_frame("Valeurs manquantes"))

# Afficher le nombre de duplcats
st.subheader("\nNombre de doublons :")
st.write(df.duplicated().sum())

# Afficher les types de données
st.subheader("\nTypes de données :")
st.write(df.dtypes.groupby(df.dtypes).size())

# Nettoyer les données en supprimant les doublons
df_cleaned = df.drop_duplicates()
st.write("\nDonnées nettoyées, doublons supprimés.")

# Analyse des contenus
- Films vs séries : proportions, tendances par année.
- Genres principaux (listed_in) : regroupement, fréquences.
- Répartition géographique (country).
- Casting & réalisateurs : analyse simple (comptages, noms fréquents).

In [ ]:
# |exporti

import pandas as pd

from streamlit_jupyter import StreamlitPatcher, tqdm

StreamlitPatcher().jupyter()

st.header("Analyse des contenus")

pd.set_option('display.max_rows', None)
df = pd.read_csv("data/netflix_titles.csv")

# Afficher le nombre total de films et son pourcentage
total_movies = df[df["type"] == "Movie"]
st.subheader("Nombre total de films :")
st.write(len(total_movies), " (", round(len(total_movies) / len(df) * 100, 2), "%)")

# Afficher le nombre total de séries et son pourcentage
total_series = df[df["type"] == "TV Show"]
st.subheader("Nombre total de séries :")
st.write(len(total_series), " (", round(len(total_series) / len(df) * 100, 2), "%)")

# Afficher le nombre de films et séries par année de sortie
st.subheader("Nombre de films et séries par année de sortie :")
pivot_df = df.pivot_table(index='type', columns='release_year', values='show_id', aggfunc='count', fill_value=0)
st.write(pivot_df)

# Afficher le nombre de films et séries par catégorie
st.subheader("Nombre de films et séries par catégorie :")
df_listed_in = df.dropna(subset=['listed_in']).copy()
df_listed_in['listed_in'] = df_listed_in['listed_in'].str.split(', ') # Séparer les catégories
df_exploded = df_listed_in.explode('listed_in') # Exploser les lignes pour chaque catégorie
category_type_counts = df_exploded.groupby(['listed_in', 'type'])['show_id'].count().unstack(fill_value=0) # Grouper et compter
categories = sorted(df_exploded['listed_in'].unique())
selected_value = st.selectbox("Sélectionner une catégorie :", options=categories, width=200) # Sélection de la catégorie
st.dataframe(
    category_type_counts.loc[[selected_value]], column_config={
        "listed_in": "Catégorie",
        "Movie": "Films",
        "TV Show": "Séries",
    }
)

# Afficher le nombre de films et séries par pays
st.subheader("Nombre de films et séries par pays :")
df_countries = df.dropna(subset=['country']).copy()
df_countries['country'] = df_countries['country'].str.split(', ') # Séparer les pays
df_exploded = df_countries.explode('country') # Exploser les lignes pour chaque pays
country_type_counts = df_exploded.groupby(['country', 'type'])['show_id'].count().unstack(fill_value=0) # Grouper et compter le nombre de films et séries par pays
countries = sorted(df_exploded['country'].unique())
selected_value = st.selectbox("Sélectionner un pays :", options=countries, width=200) # Sélection du pays
st.dataframe(
        country_type_counts.loc[[selected_value]], column_config={
            "country": "Pays",
            "Movie": "Films",
            "TV Show": "Séries",
        }
    )

# Afficher les 10 acteurs les plus fréquents
st.subheader("Les acteurs les plus fréquents :")
selected_number = st.number_input("Nombre d'acteurs à afficher :", min_value=1, max_value=50, value=10, key="num_actors", width=200)
df_cast = df.dropna(subset=['cast']).copy()
df_cast['cast'] = df_cast['cast'].str.split(', ') # Séparer les acteurs
df_exploded = df_cast.explode('cast') # Exploser les lignes pour chaque acteur
actor_counts = df_exploded['cast'].value_counts().head(selected_number).to_frame() # Compter les acteurs les plus fréquents
st.dataframe(
    actor_counts, column_config={
        "cast": "Acteur",
        "count": "Nombre d'apparitions",
    }
)

# Afficher les 10 réalisateurs les plus fréquents
st.subheader("Les réalisateurs les plus fréquents :")
selected_number = st.number_input("Nombre de réalisateurs à afficher :", min_value=1, max_value=50, value=10, key="num_directors", width=200)
df_directors = df.dropna(subset=['director']).copy()
df_directors['director'] = df_directors['director'].str.split(', ') # Séparer les réalisateurs
df_exploded = df_directors.explode('director') # Exploser les lignes pour chaque réalisateur
director_counts = df_exploded['director'].value_counts().head(selected_number).to_frame() # Compter les réalisateurs les plus fréquents
st.dataframe(
    director_counts, column_config={
        "director": "Réalisateur",
        "count": "Nombre de réalisations",
    }
)

# Analyse temporelle
- Distribution par année de sortie (release_year).
- Analyse de la colonne date_added.

# Visualisation
- Représentations obligatoires :
- histogrammes, countplots, boxplots (Seaborn / Matplotlib)
- visualisations interactives (Plotly)
- Choix libres : nuages de mots, diagrammes circulaires, timelines, etc.

# Synthèse
- Résumé des observations principales.
- Tendances remarquables (contenus récents, pays dominants, genres populaires).

# Exportation du notebook 

In [ ]:
from nbdev.export import nb_export
nb_export('project.ipynb', lib_path='./app/', name='netflix_analysis')